### Unit 1 Homework:  Scraping the Yelp Website

Welcome!  For this homework assignment you'll be tasked with building a web scraper in a manner that builds on what was covered in our web scraping class.

The assignment will extend the lab work done during that time, where we built a dataset that listed the name, number of reviews and price range for restaurant on the following web page: https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1

**What You'll Turn In:**

A finished jupyter notebook that walks us through the steps you took in order to get your results.  Provide notes where appropriate to explain what you are doing.

The notebook should produce a finished dataset at the end.  

If for some reason you're experiencing problems with the final result, please let someone know when turning it in.
 
Your homework will be divided into three tiers, each of which have increasing levels of difficulty:

##### Tier 1: Five Columns From the First Page

At the most basic level for this assignment, you will need to extend what we did in class, and create a dataset that has five columns in it that are 30 rows long.  This means you will not need to go off the first page in order to complete this section.

##### Tier 2:  100 Row Dataset With At Least 3 Columns

For this portion of the assignment, take 3 of your columns from step 1, and extend them out to multiple pages on the yelp website.  You should appropriately account for the presence of missing values.

##### Tier 3:  100 Row Dataset With At Least 5 Columns

Very similar to Tier 2, but if you use this many columns you will be forced to encounter some columns that will frequently have missing values, whereas with Tier 2 you could likely skip these if you wanted to.  

##### Tier 4:  100 Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Restaurants often have different categories associated with them, so grabbing them individually as separate values is often challenging.  To complete this tier, you'll have to find a way to 'pick out' each of the individual categories as their own separate column value.  

##### Tier 5:  Unlimited Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Take what you did in Tier 4, and extend it so that the code will work with an arbitrary number of pages.  Ie, regardless of how many pages there are listing the best restaurants in London, your scraper will find them, and cleanly parse their information into clean datasets.

### Hints

Here are a few tips that will save you time when completing this assignment:

 - The name, average rating, total ratings and neighborhood of a restaurant tend to be the 'easy' ones, because they rarely have missing values, so what ever logic you use on the first page will typically apply to all pages.  They are a good place to start
 - Phone numbers, price ranges and reviews are more commonly missing, so if you are trying to get a larger number of items from them across multiple pages you should expect to do some error handling
 - You can specify any sort of selector when using the `find_all()` method, not just `class`.  For example, imagine you have the following `<div>` tag:
    `<div class='main-container red-blue-green' role='front-unit' aria-select='left-below'>Some content here</div>`
    
   This means that when you use `scraper.find_all('div')`, you can pass in arguments like `scraper.find_all('div', {'role': 'front-unit'})` or anything else that allows you to isolate that particular tag.
 - When specifying selectors like `{'class': 'dkght__384Ko'}`, sometimes less is more.  If you include multiple selectors, you are saying return a tag with **any one of these** distinctions, not all of them.  So if your results are large, try different combinations of selectors to get the smallest results possible.
 - If you begin dealing with values that are unreliably entered, you should use the 'outside in' technique where you grab a parent container that holds the element and find a way to check to see if a particular value is there by scraping it further.  The best way to do this is to try and find a unique container for every single restaurant.  This means that you will have a reliable parent element for every single restaurant, and within *each of these* you can search for `<p>`, `<a>`, `<div>`, and `<span>` tags and apply further logic.
 - When you get results from `BeautifulSoup`, you will be given data that's denoted as either `bs4.element.Tag` or `bs4.element.ResultSet`.  They are **not the same**.  Critically, you can search a `bs4.element.Tag` for further items, but you cannot do this with a `bs4.element.ResultSet`.  
 
   For example, let's say you grab all of the divs from a page with `scraper.find_all('div')` and save it as the variable `total_divs`.  This means `total_divs` will look somethig like this:  
   
   `[<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>]`
      
   In this case the variable `total_divs` is a result set and there's nothing else you can do to it directly.  However, every item within `total_divs` is a tag, which means you can scrape it further.  
   
   So if you wanted you could write a line like:  `total_paragraphs = [div.find_all('p') for div in total_divs]`, and get the collection of paragraphs within each div.  
   
   If you confuse the two you'll get the following error message:  
   
   `AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?`
 - The values of the different selectors change periodically on yelp, so if your scraper all of a sudden stops working that's probably why.  Ie, if you have a command like `scraper.find_all('div', {'class': '485dk0W__container09'}` that no longer returns results, the class `485dk0W__container09` may now be `r56kW__container14` or something similar.

In [1]:
# import libraries
import requests
import pandas as pd
import bs4

In [ ]:
# I attempted all 5 tiers at once, so the steps are combined

In [2]:
# Step 1: Pulling data from all pages on the website
# using a while loop to pull all restaurant containers as lists. If the page is blank, the list is null and the pull stops.


# Counter for page number on the url- starts at 0 and increments of 10
item_counter=0
test_list=['start']

# these are the lists that the cleaned data will go in
restaurant_data_all=[]
address_data_all=[]

# Main container for each restaurant.This is a <div> class. It contains the <div> and <p> classes that we need
ex_classes={'class': 'container__09f24__21w3G'}

# within the main container, these are the <span> class items that we need, for restaurant data
test_classes={
    'class':'text__09f24__2tZKC',
    'class':'text-color--black-extra-light__09f24__38DtK',
    'class':'text-align--left__09f24__3Drs0' #restaurant name, price, reviews and tags
    }

# within the main container, these are the <p> class items that we need, for address data
aa_test={'class':'text-size--small__09f24__1Z_UI' #phone, address and area, p type
        }



while len(test_list)>0:
    raw_data=requests.get(f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&start={item_counter}')
    scraper=bs4.BeautifulSoup(raw_data.text)
    
    # get the data in the main container, in list form
    yex_data=scraper.find_all('div',ex_classes)
    aa=[data.find_all(['p','div']) for data in yex_data]
    
    # this is the while-loop test: if this is null the pull stops
    test_list=aa
    
    for item in aa:
        restaurant_data=[]
        address_data=[]
        el=item[0]
        bb=el.find_all('span',test_classes)
        for element in bb:
            restaurant_data.append(element.text)          
        #print(restaurant_data)
        cc=el.find_all('p',aa_test)
        for piece in cc:
            #print(piece.text)
            address_data.append(piece.text)
        #print(address_data) 
        restaurant_data_all.append(restaurant_data)
        address_data_all.append(address_data)
    
    item_counter=item_counter+10 
            


In [3]:
#checking for completeness of data
print("size_restaurant_data: ",len(restaurant_data_all))
print("size_address_data: ",len(address_data_all))

size_restaurant_data:  240
size_address_data:  240


In [4]:
# We now have two lists of nested list data:
# 1. restaurant details like name, price, reviews and tags
# 2. address details- phone, address, location

# Next step: break out the data into 'f_' lists to go into the final dataframe
# Each item in the restaurant list corresponds to a single restaurant. An empty list is created for expected datapoints.
#     if the expected datapoint doesnt exist,the final list will contain an empty nested list there
#     tags data can have multiple string values, so that list may contain multiple elements

f_names_list_all=[]
f_reviews_list_all=[]
f_price_list_all=[]
f_tags_list_all=[]

for item in restaurant_data_all:
    #print(item)
    f_names_list=[]
    f_reviews_list=[]
    f_price_list=[]
    f_tags_list=[]
    for element in item:
        if element[0].isdigit() and '\xa0' in element:
            f_name=' '
            f_name=f_name.join(element.split()[1:])
            f_names_list.append(f_name) 
            
            
        elif element.isdigit():
            f_reviews=element
            f_reviews_list.append(f_reviews)
        elif '\xa3' in element:
            f_price=element
            f_price_list.append(f_price)
        elif '\xa0' not in element:
            f_tag=element.replace(', ','')
            f_tags_list.append(f_tag)
            
    f_names_list_all.append(f_names_list)   
    f_reviews_list_all.append(f_reviews_list)
    f_price_list_all.append(f_price_list)
    f_tags_list_all.append(f_tags_list)
    





In [5]:
#checking for completeness of final dataset:
print("size_names_data: ", len(f_names_list_all))
print("size_reviewscount_data: ", len(f_reviews_list_all))
print("size_pricerange_data: ", len(f_price_list_all))
print("size_tags_data: ", len(f_tags_list_all))

size_names_data:  240
size_reviewscount_data:  240
size_pricerange_data:  240
size_tags_data:  240


In [6]:
#similarly, for address data
f_phone_list_all=[]
f_address_list_all=[]
f_location_list_all=[]

for item in address_data_all:
    #print(item)
    f_phone_list=[]
    f_address_list=[]
    f_location_list=[]
    
    for element in item:
        if element[0:3]=='020':
            f_phone_list.append(element)
        elif element[0].isdigit():
            f_address_list.append(element)
        else:
            f_location_list.append(element)
            
    f_phone_list_all.append(f_phone_list)   
    f_address_list_all.append(f_address_list)
    f_location_list_all.append(f_location_list)






In [7]:
#testing completeness of dataset
print('size_phone_data: ',len(f_phone_list_all))
print('size_address_data: ',len(f_address_list_all))
print('size_location_data: ',len(f_location_list_all))


size_phone_data:  240
size_address_data:  240
size_location_data:  240


In [8]:
# the data is now in a list of lists. Next Step: create some new lists with values

f_names_all=[]
for item in f_names_list_all:
    if item==[]:
        f_names_all.append('NA')
    else:
        f_names_all.append(item[0])


f_price_all=[]
for item in f_price_list_all:
    if item==[]:
        f_price_all.append('NA')
    else:
        f_price_all.append(item[0])


f_reviews_all=[]
for item in f_reviews_list_all:
    if item==[]:
        f_reviews_all.append('NA')
    else:
        f_reviews_all.append(item[0])


f_tags_all=[]
for item in f_tags_list_all:
    if item==[]:
        f_tags_all.append('NA')
    else:
        f_tags_all.append(item)

f_phone_all=[]
for item in f_phone_list_all:
    if item==[]:
        f_phone_all.append('NA')
    else:
        f_phone_all.append(item[0])
    
        
f_location_all=[]
for item in f_location_list_all:
    if item==[]:
        f_location_all.append('NA')
    else:
        f_location_all.append(item[0])
        

f_address_all=[]
for item in f_address_list_all:
    if item==[]:
        f_address_all.append('NA')
    else:
        f_address_all.append(item[0])
        

# splitting the tags list into multiple columns  
# How many columns to do i need to create for category tags?
# max(len(item) for item in f_tags_list_all)
category_tag1=[]
category_tag2=[]
category_tag3=[]
for item in f_tags_list_all:
    if len(item)==0:
        category_tag1.append('NA')
        category_tag2.append('NA')
        category_tag3.append('NA')
    elif len(item)==1:
        category_tag1.append(item[0])
        category_tag2.append('NA')
        category_tag3.append('NA')
    elif len(item)==2:
        category_tag1.append(item[0])
        category_tag2.append(item[1])
        category_tag3.append('NA')
    elif len(item)==3:
        category_tag1.append(item[0])
        category_tag2.append(item[1])
        category_tag3.append(item[2])

In [9]:
yelp_data=pd.DataFrame({'RestaurantName':f_names_all,
                        'PriceRange': f_price_all,
                        'ReviewCount': f_reviews_all,
                        #'Tags': f_tags_all,
                        'Category1': category_tag1,
                        'Category2': category_tag2,
                        'Category3': category_tag3,
                        'Address': f_address_all,
                        'Location': f_location_all,
                        'PhoneNumber': f_phone_all
                       })
yelp_data
        

    

,RestaurantName,PriceRange,ReviewCount,Category1,Category2,Category3,Address,Location,PhoneNumber
0,The Mayfair Chippy,££,280,Fish & Chips,NA,NA,14 North Audley Street,Mayfair,020 7741 2233
1,Ffiona’s Restaurant,££,269,British,NA,NA,51 Kensington Church Street,Kensington,020 7937 4152
2,Dishoom,££,1840,Indian,NA,NA,12 Upper Saint Martin's Lane,Covent Garden,020 7420 9320
3,The Golden Chippy,££,107,Fish & Chips,NA,NA,62 Greenwich High Road,Deptford,020 8692 4333
4,Padella,££,205,Italian,NA,NA,6 Southwark Street,London Bridge,NA
...,...,...,...,...,...,...,...,...,...
235,Eat Aroi Thai,££,23,Thai,NA,NA,100 N End Road,Barons Court,020 7381 6222
236,La Mian & Dim Sum Stall,£,6,Street Vendors,Dim Sum,NA,NA,Brick Lane Market,NA
237,BunBunBun Vietnamese Food,££,33,Vietnamese,NA,NA,134B Kingsland Road,Haggerston,020 7729 6494
238,Orjowan Lebanese Cuisine,££,105,Middle Eastern,NA,NA,6 Kenway Road,South Kensington,020 7370 3074


In [11]:
yelp_data.head(15)

,RestaurantName,PriceRange,ReviewCount,Category1,Category2,Category3,Address,Location,PhoneNumber
0,The Mayfair Chippy,££,280,Fish & Chips,NA,NA,14 North Audley Street,Mayfair,020 7741 2233
1,Ffiona’s Restaurant,££,269,British,NA,NA,51 Kensington Church Street,Kensington,020 7937 4152
2,Dishoom,££,1840,Indian,NA,NA,12 Upper Saint Martin's Lane,Covent Garden,020 7420 9320
3,The Golden Chippy,££,107,Fish & Chips,NA,NA,62 Greenwich High Road,Deptford,020 8692 4333
4,Padella,££,205,Italian,NA,NA,6 Southwark Street,London Bridge,NA
5,Flat Iron,££,380,Steakhouses,NA,NA,17 Beak Street,Soho,NA
6,The Queens Arms,££,118,British,Pubs,Gastropubs,11 Warwick Way,Victoria,020 7834 3313
7,The Fat Bear - Temp. CLOSED,££,122,American (Traditional),Soul Food,Cajun/Creole,61 Carter Lane,Blackfriars,020 7236 2498
8,Mother Mash,££,468,British,NA,NA,26 Ganton Street,Soho,020 7494 9644
9,The Grazing Goat,££,242,Gastropubs,British,NA,6 New Quebec Street,Marylebone,020 7724 7243
